In [1]:
from melo.api import TTS
from IPython.display import Audio

# Speed is adjustable
speed = 1.0

# CPU is sufficient for real-time inference.
# You can set it manually to 'cpu' or 'cuda' or 'cuda:0' or 'mps'
device = 'cuda' # Will automatically use GPU if available

# English
model = TTS(language='EN', device=device)
speaker_ids = model.hps.data.spk2id



C:\Users\MYSTIC Ganesh\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


None


C:\Users\MYSTIC Ganesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


None


In [2]:
# American accent
text = """
You will be able to reuse this secret in all of your notebooks.
"""
output_path = 'en-us.wav'
model.tts_to_file(text, speaker_ids['EN-US'], output_path, speed=speed)


# Load the wav file
audio_file_path = "en-us.wav"

# Display the audio player
Audio(audio_file_path, autoplay=True)

#23 seconds -> cpu
#3 seconds -> gpu

 > Text split to sentences.
You will be able to reuse this secret in all of your notebooks.
 > ===========================


  0%|          | 0/1 [00:00<?, ?it/s]Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1/1 [00:04<00:00,  4.44s/it]


In [4]:
# American accent
text = """
You will be able to reuse this secret in all of your notebooks. Don't you make a fool with your money now.
"""
output_path = 'en-us.wav'
sound = model.tts_to_file(
    text=text, speaker_id=speaker_ids['EN-US'], speed=speed)


 > Text split to sentences.
You will be able to reuse this secret in all of your notebooks. Don't you make a fool with your money now.
 > ===========================


100%|██████████| 1/1 [00:00<00:00,  2.01it/s]


In [5]:
import io
import soundfile as sf
buffer = io.BytesIO()
sf.write(buffer, sound, 48000, format='WAV')
buffer.seek(0)
# now we can send the buffer as api response instead of saving as file

0

In [ ]:
import io
import time
import queue
import threading
from flask import Flask, request, Response
import soundfile as sf
import numpy as np

app = Flask(__name__)

# Queue to hold processed audio chunks
audio_queue = queue.Queue()


def generate_audio_stream():
    while True:
        chunk = audio_queue.get()
        if chunk is None:
            break
        yield chunk


def audio_numpy_concat(audio_list, sr, speed):
    # Example function to concatenate audio arrays with a given sample rate
    return np.concatenate(audio_list)


@app.route('/process_audio', methods=['POST'])
def process_audio():
    if 'audio' not in request.files:
        return "No audio file part", 400

    files = request.files.getlist('audio')

    if not files or any(file.filename == '' for file in files):
        return "No selected file", 400

    def process_files():
        audio_list = []
        for file in files:
            audio_data, sample_rate = sf.read(file)
            audio_list.append(audio_data)

        # Concatenate all audio chunks
        concatenated_audio = audio_numpy_concat(
            audio_list, sr=48000, speed=1.0)

        # Save the concatenated audio to an in-memory buffer
        buffer = io.BytesIO()
        sf.write(buffer, concatenated_audio, 48000, format='WAV')
        buffer.seek(0)

        # Put the concatenated audio in the queue
        audio_queue.put(buffer.read())

        # Indicate that processing is done
        audio_queue.put(None)

    # Start a thread to process the files
    threading.Thread(target=process_files).start()

    return Response(generate_audio_stream(), mimetype='audio/wav')


if __name__ == '__main__':
    app.run(debug=True)
